# Decision Tree (xgboosted)

### Predictors: Numerical Variables + Useful Variables from Clustering/Correlation (including Fedu)

In [1]:
# Basic Libraries
#!pip install xgboost
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

In [2]:
student_por = pd.read_csv('student-por.csv',sep=';')
student_por.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000
mean,16.744222,2.514638,2.306626,1.568567,1.930663,0.221880,3.930663,3.180277,3.184900,1.502311,2.280431,3.536210,3.659476,11.399076,11.570108,11.906009
std,1.218138,1.134552,1.099931,0.748660,0.829510,0.593235,0.955717,1.051093,1.175766,0.924834,1.284380,1.446259,4.640759,2.745265,2.913639,3.230656
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,2.000000,1.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,2.000000,0.000000,10.000000,10.000000,10.000000
50%,17.000000,2.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,2.000000,11.000000,11.000000,12.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,6.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,32.000000,19.000000,19.000000,19.000000


In [3]:
student_por.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      649 non-null    object
 1   sex         649 non-null    object
 2   age         649 non-null    int64 
 3   address     649 non-null    object
 4   famsize     649 non-null    object
 5   Pstatus     649 non-null    object
 6   Medu        649 non-null    int64 
 7   Fedu        649 non-null    int64 
 8   Mjob        649 non-null    object
 9   Fjob        649 non-null    object
 10  reason      649 non-null    object
 11  guardian    649 non-null    object
 12  traveltime  649 non-null    int64 
 13  studytime   649 non-null    int64 
 14  failures    649 non-null    int64 
 15  schoolsup   649 non-null    object
 16  famsup      649 non-null    object
 17  paid        649 non-null    object
 18  activities  649 non-null    object
 19  nursery     649 non-null    object
 20  higher    

### The ordinal variables are converted from integer data type to object data type, as they are categorical.

In [4]:
student_por['failures'] = student_por['failures'].astype(float)
student_por['absences'] = student_por['absences'].astype(float)
student_por['G1'] = student_por['G1'].astype(float)
student_por['G2'] = student_por['G2'].astype(float)
student_por['G3'] = student_por['G3'].astype(float)

In [5]:
student_por['Medu'] = student_por['Medu'].astype(object)
student_por['Fedu'] = student_por['Fedu'].astype(object)
student_por['traveltime'] = student_por['traveltime'].astype(object)
student_por['studytime'] = student_por['studytime'].astype(object)
student_por['famrel'] = student_por['famrel'].astype(object)
student_por['freetime'] = student_por['freetime'].astype(object)
student_por['goout'] = student_por['goout'].astype(object)
student_por['Dalc'] = student_por['Dalc'].astype(object)
student_por['Walc'] = student_por['Walc'].astype(object)
student_por['health'] = student_por['health'].astype(object)

In [6]:
tempDF = pd.DataFrame(student_por[['Medu', 'Mjob','Fedu' ,'studytime', 'higher', 'internet', 'Dalc', 'Walc', 'failures', 'absences', 'G1', 'G2']]) #predictor
G3obj = pd.DataFrame((student_por['G3'] >= 10).replace({True: 1, False: 0})) #response

In [7]:
tempDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Medu       649 non-null    object 
 1   Mjob       649 non-null    object 
 2   Fedu       649 non-null    object 
 3   studytime  649 non-null    object 
 4   higher     649 non-null    object 
 5   internet   649 non-null    object 
 6   Dalc       649 non-null    object 
 7   Walc       649 non-null    object 
 8   failures   649 non-null    float64
 9   absences   649 non-null    float64
 10  G1         649 non-null    float64
 11  G2         649 non-null    float64
dtypes: float64(4), object(8)
memory usage: 61.0+ KB


### One-hot encoding is used to convert the nominal and ordinal variables into binary variables, so that they can be put into the decision tree as integer data type, along with the numeric variables.

In [8]:
objDF = pd.DataFrame(tempDF.select_dtypes(include=['object']))
numDF = pd.DataFrame(tempDF.select_dtypes(include=['int64', 'float64']))
predictors = pd.DataFrame()

for obj in objDF:
    col = objDF[obj].unique()
    for i in col:
        predictors[obj+" "+str(i)] = (objDF[obj]==i).replace({True:'1', False: '0'})
        predictors[obj+" "+str(i)] = (objDF[obj]==i).astype(int)
        
for num in numDF:
    predictors[num] = numDF[num]
        
predictors

,Medu 4,Medu 1,Medu 3,Medu 2,Medu 0,Mjob at_home,Mjob health,Mjob other,Mjob services,Mjob teacher,...,Dalc 4,Walc 1,Walc 3,Walc 2,Walc 4,Walc 5,failures,absences,G1,G2
0,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0.0,4.0,0.0,11.0
1,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0.0,2.0,9.0,11.0
2,0,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0.0,6.0,12.0,13.0
3,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0.0,0.0,14.0,14.0
4,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0.0,0.0,11.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1.0,4.0,10.0,11.0
645,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0.0,4.0,15.0,15.0
646,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0.0,6.0,11.0,12.0
647,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0.0,6.0,10.0,10.0


In [9]:
predictors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Medu 4         649 non-null    int32  
 1   Medu 1         649 non-null    int32  
 2   Medu 3         649 non-null    int32  
 3   Medu 2         649 non-null    int32  
 4   Medu 0         649 non-null    int32  
 5   Mjob at_home   649 non-null    int32  
 6   Mjob health    649 non-null    int32  
 7   Mjob other     649 non-null    int32  
 8   Mjob services  649 non-null    int32  
 9   Mjob teacher   649 non-null    int32  
 10  Fedu 4         649 non-null    int32  
 11  Fedu 1         649 non-null    int32  
 12  Fedu 2         649 non-null    int32  
 13  Fedu 3         649 non-null    int32  
 14  Fedu 0         649 non-null    int32  
 15  studytime 2    649 non-null    int32  
 16  studytime 3    649 non-null    int32  
 17  studytime 1    649 non-null    int32  
 18  studytime 

In [10]:
G3obj

,G3
0,1
1,1
2,1
3,1
4,1
...,...
644,1
645,1
646,0
647,1


## Building the model

In [11]:
# Import essential models and functions from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Split the Dataset into Train and Test
pred_train, pred_test, G3_train, G3_test = train_test_split(predictors, G3obj, test_size = 0.25)

# Check the sample sizes
print("Train Set :", G3_train.shape, pred_train.shape)
print("Test Set  :", G3_test.shape, pred_test.shape)

Train Set : (486, 1) (486, 37)
Test Set  : (163, 1) (163, 37)


In [22]:
# split data into X and y
X = pred_train
Y = G3_train
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 94.41%
